In [ ]:
# import pandas as pd
# import pickle
# import os

# # Load CSV
# df = pd.read_csv(r"D:\food_recommender\data\Food Ingredients and Recipe Dataset with Cleaned Ingredients.csv")

# # Replace NaN
# df = df.fillna("")

# image_folder = r"D:\food_recommender\data\Food Images"

# df["Image_Path"] = df["Image_Name"].apply(lambda x: os.path.join(image_folder, f"{x}.jpg"))

# # Create embeddings folder if missing
# os.makedirs(r"D:\food_recommender\embeddings", exist_ok=True)

# # Build meta dictionary
# meta = {
#     "title": df["Title"].tolist(),
#     "ingredients": df["Ingredients"].tolist(),
#     "cleaned_ingredients": df["Cleaned_Ingredients"].tolist(),
#     "instructions": df["Instructions"].tolist(),
#     "image_path": df["Image_Path"].tolist(),
# }

# # Save meta.pkl
# with open(r"D:\food_recommender\embeddings\meta.pkl", "wb") as f:
#     pickle.dump(meta, f)

# print("meta.pkl created successfully!")



meta.pkl created successfully!


In [9]:
import torch
from transformers import CLIPModel, CLIPProcessor
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
text_embs = []
df = pd.read_csv(r"D:\food_recommender\data\cleaned_df.csv")

df["cleaned_ingredients"] = df["cleaned_ingredients"].fillna("random text")
# The column that contains text for CLIP

TEXT_COL = "cleaned_ingredients"    # <-- change to your actual column name

# Find rows where the text column is NOT a string
bad_rows = df[~df[TEXT_COL].apply(lambda x: isinstance(x, str))]

print("Number of bad rows:", len(bad_rows))
print(bad_rows.head())  # show first few

Number of bad rows: 0
Empty DataFrame
Columns: [Unnamed: 0, title, ingredients, cleaned_ingredients, instructions, image_path]
Index: []


In [10]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


texts = df["cleaned_ingredients"].tolist()
MAX_LEN = 77  # CLIP limit

for text in tqdm(texts, desc="Embedding text"):

    # ---- Tokenize first so we can truncate ----
    tokens = processor.tokenizer(
        text,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

    with torch.no_grad():
        emb = model.get_text_features(
            input_ids=tokens["input_ids"],
            attention_mask=tokens["attention_mask"]
        )

    text_embs.append(emb.cpu().numpy()[0])

text_embs = np.array(text_embs)


print("DONE — text embeddings + meta saved")


Embedding text: 100%|██████████| 13471/13471 [25:35<00:00,  8.78it/s]

DONE — text embeddings + meta saved


In [11]:
os.makedirs(r"D:\food_recommender\embeddings", exist_ok=True)

np.save(r"D:\food_recommender\embeddings\text_embeddings_cleaned.npy", text_embs)


In [12]:
import faiss
import numpy as np

# Load embeddings
embeds = np.load(r"D:/food_recommender/embeddings/text_embeddings_cleaned.npy").astype("float32")

d = embeds.shape[1]        # dimension của embedding
index = faiss.IndexFlatL2(d)  # index L2
index.add(embeds)             # add embeddings theo thứ tự meta.pkl

# Save FAISS index
faiss.write_index(index, r"D:/food_recommender/embeddings/faiss_text_index_cleaned.bin")
